# HW1: Тональный классификатор
### Такташева Катя

In [3]:
import re
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import time
import random
import os
import json
import selenium
from selenium import webdriver

session = requests.session()

### Get films' ids:

In [308]:
def get_film_id(film):
    link = re.search('href="/film/(\d+?)/"', str(film)).group(1)
    return link


def get_films(pn):
    ids = []
    page = session.get(f'https://www.kinopoisk.ru/popular/films/?page={pn}&tab=all').text
    time.sleep(random.uniform(1.1, 5.2))
    soup = BeautifulSoup(page, 'html.parser')
    films = soup.find_all('div', {'class':'desktop-rating-selection-film-item'})
    
    for film in films:
        ids.append(get_film_id(film))
        
    return ids

In [10]:
from tqdm.auto import tqdm 

In [317]:
films = []
for pn in tqdm(range(1, 35)):
    time.sleep(random.uniform(1.1, 8.2))
    ids = get_films(pn)
    films.extend(ids)

In [318]:
len(films)

1000

### Get reviews

In [319]:
URL = 'https://www.kinopoisk.ru/film/326/reviews'
СHROME_PATH = os.path.join(os.getcwd(), 'chromedriver')
DRIVER = webdriver.Chrome(executable_path=СHROME_PATH)
SLEEP_TIME = 3

In [320]:
link = 'https://www.kinopoisk.ru/film/{}/reviews'

In [321]:
def get_reviews(link, id):
    film_reviews = {'film_id': id,
                   'pos': [],
                   'neg': []}
    plink = link.format(id)
    page = DRIVER.get(plink)
    time.sleep(random.uniform(1.1, 5.2))
    for tp in ['pos', 'neg']:
        try:
            DRIVER.find_element_by_class_name(tp).find_element_by_tag_name('a').click()
            time.sleep(random.uniform(1.1, 5.2))
            reviews = DRIVER.find_elements_by_class_name('_reachbanner_')
            for idx, rev in enumerate(reviews):
                film_reviews[tp].append(rev.text)
        except:  # если нет отзывов
            pass
    return film_reviews

In [322]:
reviews = []
for film_id in tqdm(films):
    data = get_reviews(link, film_id)
    reviews.append( data)
DRIVER.close()

Save data:

In [323]:
with open('reviews.json', 'w', encoding='utf-8') as fid:
    fid.write(json.dumps(reviews))

### Split data

In [4]:
with open('reviews.json', 'r', encoding='utf-8') as fid:
    reviews = json.loads(fid.read())

In [5]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

Using Spacy for Russian to tokenize:

In [6]:
from spacy.lang.ru import Russian
from spacy_russian_tokenizer import RussianTokenizer, MERGE_PATTERNS, SYNTAGRUS_RARE_CASES
nlp = Russian()
russian_tokenizer = RussianTokenizer(nlp, MERGE_PATTERNS + SYNTAGRUS_RARE_CASES)
nlp.add_pipe(russian_tokenizer, name='russian_tokenizer')

In [7]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [8]:
def lemmatize(text):
    words = []
    for word in nlp(text):
        lemma = morph.parse(word.text)[0].normal_form
        words.append(lemma)
    return ' '.join(words)


def split_data(data):
    new_data = []
    for film in tqdm(data):
        for review in film['pos']:
            new_data.append((film['film_id'], lemmatize(review), 'pos'))
        for review in film['neg']:
            new_data.append((film['film_id'], lemmatize(review), 'neg'))
            
    new_data = shuffle(pd.DataFrame(np.array(new_data), columns=['film_id', 'text', 'class']))
    
    train, test = np.split(new_data.sample(frac=1), [int(.80*len(new_data))])
    
    return train, test

In [11]:
train, test = split_data(reviews)

In [12]:
print(f'Train sentences: {len(train)}')
print(f'Test sentences: {len(test)}')

Train sentences: 11494
Test sentences: 2874


In [14]:
train.head()

,film_id,text,class
6167,103734,"я стыдно признаться , что данный мультипликаци...",pos
12660,363,"плохо понимать , как писать рецензия на этот ф...",pos
8379,491724,это уже восьмой фильм от режиссёр дэвид финчер...,pos
3090,1164484,в маленький город на граница безымянный импери...,pos
5709,104992,довольно странно писать свой мнение о фильм бо...,neg


In [13]:
train['class'].value_counts()

pos    6458
neg    5036
Name: class, dtype: int64

In [15]:
test['class'].value_counts()

pos    1607
neg    1267
Name: class, dtype: int64

### Create tonality dictionary

In [24]:
from collections import Counter
from nltk.corpus import stopwords
ru_stopwords = set(stopwords.words('russian'))

In [25]:
def preprocess(text, lemmatize=False):
    words = []
    for word in text.split():
        if word.isalpha() and word not in ru_stopwords:
            if lemmatize:
                word = morph.parse(word)[0].normal_form
            words.append(word)
    return words

In [26]:
def create_dict(pos_words, neg_words):
    tone_dict = {}
    for word in pos_words:
        tone_dict[word] = 'pos'
    for word in neg_words:
        tone_dict[word] = 'neg'
        
    return tone_dict


def get_tone_words(data, min_count=10):
    
    dic = {'pos': Counter(),
           'neg': Counter()}
    
    print('Computing frequency dict...')
    for idx, review in tqdm(data.iterrows(), total=len(data)):
        dic[review['class']] += Counter(preprocess(review['text']))
            
    negative = set([i[0] for i in dic['neg'].most_common() if i[1] >= min_count])
    positive = set([i[0] for i in dic['pos'].most_common() if i[1] >= min_count])
    
    intersect = positive.intersection(negative)
    for i in intersect:
        positive.discard(i)
        negative.discard(i)
    
    #min_size = min((len(positive), len(negative)))  # по-хорошему надо сравнять размеры классов, но
    #positive = list(positive)[:min_size]            # т.к. у нас довольно мало данных это сильно уменьшает
    #negative = list(negative)[:min_size]            # размер словаря и => точность классификатора
    
    print(f'No intersection: {set(positive).isdisjoint(set(negative))}')
    print(f'Positive: {len(positive)}')
    print(f'Negative: {len(negative)}')
                   
    return create_dict(positive, negative)

In [38]:
tone_dict = get_tone_words(train, 50)

Computing frequency dict...



No intersection: True
Positive: 1066
Negative: 176


### Classify

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
def classify_review(review):
    review_class = Counter()
    words = preprocess(review)
    for word in words:
        if word in tone_dict:
            review_class[tone_dict[word]] += 1 
    return review_class.most_common()[0][0] if len(review_class) > 0 else 'pos'


def make_predictions(test):
    predictions = []
    for idx, x in test.iterrows():
        pred = classify_review(x['text'])
        predictions.append(pred)
    return predictions

In [41]:
prediction = make_predictions(test[['film_id', 'text']])
print(accuracy_score(prediction, test[['class']]))

0.592553931802366


### Improvements

Самое простое:
1. **Больше данных**
    - Сейчас данных довольно мало и, поскольку (например, в сравнении с задачей по определению языка) слова менее distinct в классах, получается маленький словарь, не все слова находятся и от этого страдает доля верных ответов :( 


2. **Баланс классов**
    - **2.1** При разделении выборки на "обучающую" и тест, для лучшего результата нужно сравнять баланс классов, чтобы не получалось, что положительных отзывов в 5 раз больше, чем отрицательных. В таком случае и в словаре тональных слов один класс будет заметно преобладать, поэтому такая классификация, как мы делаем, будет с большей вероятностью выдавать именно преобладающий класс (см. ниже)
    - **2.2** Эту проблему можно решить и при создании самого словаря: если мы выравниваем размеры уже готового словаря, то тогда у нас не будет перевеса одного класса, но тут снова упираемся в проблему 1: для этого надо больше данных

3. **Подбор гиперпараметров**
    - Еще один способ для улучшения качества - подбор подходящих гиперпараметров, в нашем случае - минимальное количество вхождения слов в словарь

In [42]:
from sklearn.utils import shuffle


def balance_classes(data):
    balanced = []
    classes = data['class'].unique().tolist()
    size = data['class'].value_counts().min()
    for cl in classes:
        balanced.append(data[data['class']==cl].sample(size))
    return shuffle(pd.concat(balanced, ignore_index=True))
        

In [43]:
train_b = balance_classes(train)
test_b = balance_classes(test)

In [44]:
train_b['class'].value_counts()

pos    5036
neg    5036
Name: class, dtype: int64

In [45]:
test_b['class'].value_counts()

neg    1267
pos    1267
Name: class, dtype: int64

In [48]:
tone_dict = get_tone_words(train_b, 100)

Computing frequency dict...



No intersection: True
Positive: 240
Negative: 204


In [49]:
prediction = make_predictions(test_b[['film_id', 'text']])
print(accuracy_score(prediction, test_b[['class']]))

0.6669297553275454
